## UCL COMP0029 Individual Project for Year 3 BSc
### Robust Robotic Grasping Utilising Touch Sensing - Baseline Approach Notebook
This notebook contains the code for developing a baseline approach to grasping using classifiers: given some combinations of tactile data, end effector poses relative to the robot hand (visual data), etc., determine whether these constraints will produce a successful/unsuccessful grasp.

### 1. Load packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import gc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA

import seaborn as sns

Set device for `PyTorch` training

In [ ]:
# Use GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

Empty PyTorch cache

In [ ]:
gc.collect()
torch.cuda.empty_cache()

Set numpy seed

In [ ]:
np.random.seed(seed=1)

### 2. Load datasets from saved .npy files

To collect data for this experiment, you can run the "Collect Sensory Data" button in the Pybullet simulation. This generates a predefined number of Gaussian grasps randomly generated from a base hand pose. Each individual grasp is considered as an individual experiment, and the data collected from this experiment is split into four, each stored in its own dataset.

For all object models used in this experiment, each object has 4 datasets which include:
- `depth_ds.npy` which stores the depth tactile data from the mounted DIGIT sensors
- `color_ds.npy` which stores the colored (RGB) version of the depth tactile data from the mounted DIGIT sensors
- `poses_ds.npy` which stores the randomly-generated 6d hand poses from the simulation
- `outcomes_ds.npy` which stores the outcomes of each random pose

In [ ]:
root = "../datasets/"
object_name = "block"      # object_name should be in [bottle, block, mug]

# Construct the relative paths of each dataset and load them into the notebook
depth_ds_file_path = root + object_name + "/depth_ds.npy"
depth_data = np.load(depth_ds_file_path)

color_ds_file_path = root + object_name + "/color_ds.npy"
color_data = np.load(color_ds_file_path)

poses_ds_file_path = root + object_name + "/poses_ds.npy"
poses_data = np.load(poses_ds_file_path)

grasp_outcomes_ds_file_path = root + object_name + "/grasp_outcomes.npy"
grasp_outcomes_data = np.load(grasp_outcomes_ds_file_path)

These datasets should all be in the form of $(N\times...)$ where $N$ is the number of examples:

In [ ]:
print(f"Shape of depth_data: {depth_data.shape}")
print(f"Shape of color_data: {color_data.shape}")
print(f"Shape of poses_data: {poses_data.shape}")
print(f"Shape of grasp_outcomes_data: {grasp_outcomes_data.shape}")

Additionally, we confirm the number of successful and unsuccessful grasps recorded. This helps us in the next section to determine how many examples we should include for each class in order to produce a balanced dataset.

In [ ]:
print(f"# of sucessesful grasps: {(grasp_outcomes_data == 1).sum()}")
print(f"# of unsuccessful grasps: {(grasp_outcomes_data == 0).sum()}")

### 3. Data preprocessing

We will train a logistic regression classifier on 3 combinations of data:
- **Tactile** only: concatenated and flattened depth and color data
- **Visual** only: 6D end effector poses consisting of position (x,y,z) and orientation (r,p,y) data
- **Both**: concatenated and flattened tactile and visual data

#### 3a. Sampling
We sample a pre-defined number of examples from each class label (successful and unsuccessful grasps) to reduce the computational cost.

In [ ]:
sample_size = 10

In [ ]:
# Function to sample n samples from the datasets
def sample_data(depth_data, color_data, poses_data, outcomes_data, no_of_examples):
    d, c, p, o = [], [], [], []
    for label in np.unique(outcomes_data):
        indices = np.where(outcomes_data == label)[0]

        for i in range(no_of_examples):
           # if np.mean(depth_data[indices[i]]) > 1e-7 and np.mean(color_data[indices[i]]) > 1e-7:
            d.append(depth_data[indices[i]])
            c.append(color_data[indices[i]])
            p.append(poses_data[indices[i]])
            o.append(outcomes_data[indices[i]])
    
    depth_data = torch.from_numpy(np.array(d))
    color_data = torch.from_numpy(np.array(c))
    poses_data = torch.from_numpy(np.array(p))
    outcomes_data = torch.from_numpy(np.array(o))
    return depth_data, color_data, poses_data, outcomes_data

Sample `sample_size` samples:

In [ ]:
depth_data, color_data, poses_data, grasp_outcomes_data = sample_data(depth_data, color_data, poses_data, grasp_outcomes_data, no_of_examples=sample_size)

In [ ]:
print(f"Shape of depth_data: {depth_data.shape}")
print(f"Shape of color_data: {color_data.shape}")
print(f"Shape of poses_data: {poses_data.shape}")
print(f"Shape of grasp_outcomes_data: {grasp_outcomes_data.shape}")
print(f"# of sucessesful grasps: {(grasp_outcomes_data == 1).sum()}")
print(f"# of unsuccessful grasps: {(grasp_outcomes_data == 0).sum()}")

#### 3b. Preprocessing

We define some reusable functions for later:

In [ ]:
def normalize(tensor):
    # Normalize & standardize each column
    mean = torch.mean(tensor, axis=0)
    std = torch.std(tensor, axis=0)
    tensor = (tensor - mean) / std
    tensor[torch.isnan(tensor)] = 0
    tensor[torch.isinf(tensor)] = 0
    return tensor


def downsample_2d_tensor(tensor, factor=2):
    return torch.nn.functional.avg_pool2d(tensor.unsqueeze(0), kernel_size=factor, stride=factor, padding=0).squeeze(0)


def downsample_3d_tensor(tensor, factor=2):
    pooling_kernel = 2
    pooling_layer = torch.nn.AvgPool2d(kernel_size=pooling_kernel, stride=pooling_kernel)
    downsampled_tensor = pooling_layer(tensor.permute(0, 3, 1, 2))
    downsampled_tensor = downsampled_tensor.permute(0, 2, 3, 1)
    
    return downsampled_tensor

Since each tactile reading (depth and color) is a pair of images (one on each finger), we concatenate them together as a single 160x240 image.

In [ ]:
depth_data = torch.cat([depth_data[:, 0, :, :], depth_data[:, 1, :, :]], dim=2)
color_data = torch.cat([color_data[:, 0, :, :, :], color_data[:, 1, :, :, :]], dim=2)
print(f"Shape of depth_data: {depth_data.shape}")
print(f"Shape of color_data: {color_data.shape}")

We then concatenate the depth and color datasets to produce the flattened tactile dataset:

In [ ]:
# depth_ds = depth_data.reshape(depth_data.shape[0], -1)
# color_ds = color_data.reshape(color_data.shape[0], -1)
# depth_ds.shape,  color_ds.shape

#### 3c. Normalising the datasets

In [ ]:
# depth_ds = normalize(depth_ds)
# color_ds = normalize(color_ds)
depth_ds = normalize(depth_data)
color_ds = normalize(color_data)
visual_ds = torch.from_numpy(np.nan_to_num(normalize(poses_data)))
visual_ds

Create the tactile dataset and the complete dataset (tactile+visual)

In [ ]:
# tactile_ds = torch.cat([depth_ds, color_ds], dim=1)
# complete_ds = torch.cat([tactile_ds, visual_ds], dim=1)
tactile_ds = torch.cat([depth_ds.unsqueeze(-1), color_ds], dim=-1)
tactile_ds = torch.nan_to_num(tactile_ds)
complete_ds = torch.cat([tactile_ds.reshape(tactile_ds.shape[0], -1), visual_ds], dim=1)
complete_ds = torch.nan_to_num(complete_ds)
tactile_ds.shape, complete_ds.shape

### 4. Dataset visualisation (optional)

Only run this section to produce figures and plots for the project report.

In [ ]:
rand_indices = np.random.choice(len(depth_data), size=10)
cols = 4
rows = 5
fig = plt.figure(figsize=(16, 14))

# Note that the plots use the "..._data" datasets instead of the "..._ds" datasets since 
# the "..._ds" datasets are already flattened for training
for i in range(rows*cols):
    fig.add_subplot(rows, cols, i+1)
    if i % 2 == 0:
        plt.xlabel("Depth")
        plt.ylabel("SUCCESS" if grasp_outcomes_data[rand_indices[i//2]].item() == 1.0 else "FAILURE")
        plt.imshow(np.array(depth_data[rand_indices[i//2]]), cmap='gray')
    elif i % 2 == 1:
        plt.xlabel("Color")
        plt.imshow(np.array(color_data[rand_indices[i//2]]) / 255., cmap='gray')
plt.show()

Plot the end effector poses to investigate if we can simply visualise a clear decision boundary between successful & unsuccessful grasps

In [ ]:
### Helper function to calculate approach vectors of a 6D pose
def calc_approach_vector(coord):
    R = np.array([[np.cos(coord[5])*np.cos(coord[4]), np.cos(coord[5])*np.sin(coord[4])*np.sin(coord[3])-np.sin(coord[5])*np.cos(coord[3]), np.cos(coord[5])*np.sin(coord[4])*np.cos(coord[3])+np.sin(coord[5])*np.sin(coord[3])],
              [np.sin(coord[5])*np.cos(coord[4]), np.sin(coord[5])*np.sin(coord[4])*np.sin(coord[3])+np.cos(coord[5])*np.cos(coord[3]), np.sin(coord[5])*np.sin(coord[4])*np.cos(coord[3])-np.cos(coord[5])*np.sin(coord[3])],
              [-np.sin(coord[4]), np.cos(coord[4])*np.sin(coord[3]), np.cos(coord[4])*np.cos(coord[3])]])
    return R.T

In [ ]:
from math import sin, cos


Z_PADDING = 0.2        # Adjust for visualisation of end poses

plot = plt.figure()
ax = plot.add_subplot(projection='3d', computed_zorder=False)
ax.axes.set_xlim3d(-0.05, 0.05)
ax.axes.set_ylim3d(-0.05, 0.05)
ax.axes.set_zlim3d(0, 0.1)

    
# Dimensions of the block object: (W=0.025, H=0.05, D=0.05)
# (0,0,0), (0,1,0), (1,0,0), (0,0,1) -> unit cube
cube_definition = [(-0.0125, -0.025, 0), (0.0125, -0.025, 0), (-0.0125, 0.0255, 0), (-0.0125, -0.025, 0.05)]
plot_cube(cube_definition, ax)
# Plot hand poses (cartesian)
# ax.scatter(poses_data[:, 0], poses_data[:, 1], poses_data[:, 2], color=['red' if g==1 else 'green' for g in grasp_outcomes_data])
for i in range(poses_data.shape[0]):
    x, y, z, alpha, beta, gamma = poses_data[i, :]
    z -= Z_PADDING * 1.65

    # create rotation matrix based on Euler angles
    Rx = np.array([[1, 0, 0], [0, cos(alpha), -sin(alpha)], [0, sin(alpha), cos(alpha)]])
    Ry = np.array([[cos(beta), 0, sin(beta)], [0, 1, 0], [-sin(beta), 0, cos(beta)]])
    Rz = np.array([[cos(gamma), -sin(gamma), 0], [sin(gamma), cos(gamma), 0], [0, 0, 1]])
    R = Rz.dot(Ry.dot(Rx))

    # calculate endpoints of line based on orientation
    vec = np.array([0, 0.015, 0])
    vec_rotated1 = R.dot(vec)
    endpoint1 = [x + vec_rotated1[0], y + vec_rotated1[1], z + vec_rotated1[2]]
    
    vec_rotated2 = R.dot(-vec)
    endpoint2 = [x + vec_rotated2[0], y + vec_rotated2[1], z + vec_rotated2[2]]

    # set midpoint to the actual point
    midpoint = [x, y, z]

    # plot line through point in orientation direction
    ax.plot([endpoint1[0], endpoint2[0]], [endpoint1[1], endpoint2[1]], [endpoint1[2], endpoint2[2]], color='red' if grasp_outcomes_data[i]==1 else 'green', zorder=20)

    # plot vertical lines starting from endpoints
    ax.plot([endpoint1[0], endpoint1[0]], [endpoint1[1], endpoint1[1]], [endpoint1[2], endpoint1[2]-0.025], color='red' if grasp_outcomes_data[i]==1 else 'green', zorder=20)
    ax.plot([endpoint2[0], endpoint2[0]], [endpoint2[1], endpoint2[1]], [endpoint2[2], endpoint2[2]-0.025], color='red' if grasp_outcomes_data[i]==1 else 'green', zorder=20)


# Draw block object
def plot_cube(cube_definition, ax):
    cube_definition_array = [np.array(list(item)) for item in cube_definition]
    points = []
    points += cube_definition_array
    vectors = [
        cube_definition_array[1] - cube_definition_array[0],
        cube_definition_array[2] - cube_definition_array[0],
        cube_definition_array[3] - cube_definition_array[0]
    ]
    points += [cube_definition_array[0] + vectors[0] + vectors[1]]
    points += [cube_definition_array[0] + vectors[0] + vectors[2]]
    points += [cube_definition_array[0] + vectors[1] + vectors[2]]
    points += [cube_definition_array[0] + vectors[0] + vectors[1] + vectors[2]]
    points = np.array(points)
    edges = [
        [points[0], points[3], points[5], points[1]],
        [points[1], points[5], points[7], points[4]],
        [points[4], points[2], points[6], points[7]],
        [points[2], points[6], points[3], points[0]],
        [points[0], points[2], points[4], points[1]],
        [points[3], points[6], points[7], points[5]]
    ]
    faces = Poly3DCollection(edges, linewidths=1, edgecolors='k')
    faces.set_facecolor((0,0,1,0.1))
    faces.set_zorder(3)
    ax.add_collection3d(faces)
    # Plot the points themselves to force the scaling of the axes
    ax.scatter(points[:,0], points[:,1], points[:,2], s=0)
    # ax.set_aspect('equal')

# Display the plot at different angles
ax.view_init(elev=15, azim=30, roll=0)

If the green and red points are mixed up together, it proves that there is a problem when generating the random poses.

### 5. Model training

We now train our Logistic Regression models on the 3 combinations of our data (tactile, visual, both):
- Raw data
- Principal Component Analysis - 2 main components
- Convolutional Neural Network processed data

#### Prepare training and testing datasets

In [ ]:
X_tactile_train, X_tactile_test, y_tactile_train, y_tactile_test = train_test_split(tactile_ds.reshape(tactile_ds.shape[0], -1), grasp_outcomes_data, test_size=0.2, random_state=0)
X_visual_train, X_visual_test, y_visual_train, y_visual_test = train_test_split(visual_ds, grasp_outcomes_data, test_size=0.2, random_state=0)
X_complete_train, X_complete_test, y_complete_train, y_complete_test = train_test_split(complete_ds, grasp_outcomes_data, test_size=0.2, random_state=0)

### 5.1 Raw data

#### 5.1.1 Raw data (tactile only) + LR

In [ ]:
model_511 = LogisticRegression(random_state=0, max_iter=400)
model_511.fit(X_tactile_train, y_tactile_train)
model_511_predictions = model_511.predict(X_tactile_test)

#### 5.1.2 Raw data (visual only) + LR

In [ ]:
model_512 = LogisticRegression(random_state=0, max_iter=400)
model_512.fit(X_visual_train, y_visual_train)
model_512_predictions = model_512.predict(X_visual_test)

#### 5.1.3 Raw data (both) + LR

In [ ]:
model_513 = LogisticRegression(random_state=0, max_iter=400)
model_513.fit(X_complete_train, y_complete_train)
model_513_predictions = model_513.predict(X_complete_test)

### 5.2 PCA

There may not be enough memory to perform the PCA calculation on the entire input tensor. One solution to this issue is to perform PCA in a mini-batch fashion, where you break down the input tensor into smaller chunks and perform PCA on each chunk separately. Then, you can concatenate the results from each chunk to get the final result.

In [ ]:
def pca_multid(tensor, k):
    tensor = tensor.reshape((tensor.shape[0], -1))
    pca = PCA(n_components=k)    
    pca.fit(tensor)
    return pca.transform(tensor)

def pca_2d(tensor, k):
    pca = PCA(n_components=k)
    pca.fit(tensor)
    return pca.transform(tensor)

In [ ]:
K = 3

pca_tactile_ds = torch.Tensor(np.array([pca_multid(i, k=3) for i in tactile_ds]))
pca_visual_ds = torch.Tensor(pca_2d(visual_ds, k=K))
pca_complete_ds = torch.Tensor(pca_2d(complete_ds, k=K))
pca_tactile_ds.shape, pca_visual_ds.shape, pca_complete_ds.shape

In [ ]:
X_pca_tactile_train, X_pca_tactile_test, y_pca_tactile_train, y_pca_tactile_test = train_test_split(pca_tactile_ds.reshape(pca_tactile_ds.shape[0], -1), grasp_outcomes_data, test_size=0.2, random_state=0)
X_pca_visual_train, X_pca_visual_test, y_pca_visual_train, y_pca_visual_test = train_test_split(pca_visual_ds, grasp_outcomes_data, test_size=0.2, random_state=0)
X_pca_complete_train, X_pca_complete_test, y_pca_complete_train, y_pca_complete_test = train_test_split(pca_complete_ds, grasp_outcomes_data, test_size=0.2, random_state=0)

#### 5.2.1 PCA (tactile only) + LR

In [ ]:
model_521 = LogisticRegression(random_state=0, max_iter=400)
model_521.fit(X_pca_tactile_train, y_pca_tactile_train)
model_521_predictions = model_521.predict(X_pca_tactile_test)

#### 5.2.2 PCA (visual only) + LR

In [ ]:
model_522 = LogisticRegression(random_state=0, max_iter=400)
model_522.fit(X_pca_visual_train, y_pca_visual_train)
model_522_predictions = model_522.predict(X_pca_visual_test)

#### 5.2.3 PCA (both) + LR

In [ ]:
model_523 = LogisticRegression(random_state=0, max_iter=400)
model_523.fit(X_pca_complete_train, y_pca_complete_train)
model_523_predictions = model_523.predict(X_pca_complete_test)

### 5.3 CNN for dimensionality reduction

In [ ]:
# A simple convolutional neural network that extracts features from an input tensor
class FeatureExtractorCNN(nn.Module):
    def __init__(self):
        super(FeatureExtractorCNN, self).__init__()
        self.conv1 = nn.Conv2d(4, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=3)


    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool(x)
        
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.pool(x)

        return x

In [ ]:
# Preprocess data using CNN feature extraction
cnn = FeatureExtractorCNN()
cnn_tactile = torch.cat([cnn(img.float().permute(2,0,1)).unsqueeze(0) for img in tactile_ds])
cnn_tactile = cnn_tactile.reshape(cnn_tactile.shape[0], -1)
cnn_tactile.shape

In [ ]:
# We simply combine the cnn-processed tactile data (from Section 5.3.1) with the visual data
cnn_complete_ds = torch.cat([cnn_tactile.reshape(cnn_tactile.shape[0], -1), visual_ds], dim=1)
cnn_complete_ds.shape

In [ ]:
X_cnn_tactile_train, X_cnn_tactile_test, y_cnn_tactile_train, y_cnn_tactile_test = train_test_split(cnn_tactile.detach().numpy(), grasp_outcomes_data, test_size=0.2, random_state=0)
X_cnn_complete_train, X_cnn_complete_test, y_cnn_complete_train, y_cnn_complete_test = train_test_split(cnn_complete_ds.detach().numpy(), grasp_outcomes_data, test_size=0.2, random_state=0)

#### 5.3.1 CNN (tactile only) + LR

In [ ]:
model_531 = LogisticRegression(random_state=0, max_iter=400)
model_531.fit(X_cnn_tactile_train, y_cnn_tactile_train)
model_531_predictions = model_531.predict(X_cnn_tactile_test)

#### 5.3.2 CNN (visual only) + LR

#### 5.3.3 CNN (both) + LR

In [ ]:
model_533 = LogisticRegression(random_state=0, max_iter=400)
model_533.fit(X_cnn_complete_train, y_cnn_complete_train)
model_533_predictions = model_533.predict(X_cnn_complete_test)

### 5.4 Results

In [ ]:
model_data = {
    "Raw (tactile only)": {"score": model_511.score(X_tactile_test, y_tactile_test), "preds": model_511_predictions, "test_set": y_tactile_test},
    "Raw (visual only)": {"score": model_512.score(X_visual_test, y_visual_test), "preds": model_512_predictions, "test_set": y_visual_test},
    "Raw (both)": {"score": model_513.score(X_complete_test, y_complete_test),"preds": model_513_predictions,"test_set": y_complete_test},
    "PCA (tactile only)": {"score": model_521.score(X_pca_tactile_test, y_pca_tactile_test),"preds": model_521_predictions,"test_set": y_pca_tactile_test},
    "PCA (visual only)": {"score": model_522.score(X_pca_visual_test, y_pca_visual_test),"preds": model_522_predictions,"test_set": y_pca_visual_test},
    "PCA (both)": {"score": model_523.score(X_pca_complete_test, y_pca_complete_test),"preds": model_523_predictions,"test_set": y_pca_complete_test},
    "CNN (tactile only)": {"score": model_531.score(X_cnn_tactile_test, y_cnn_tactile_test), "preds": model_531_predictions,"test_set": y_cnn_tactile_test},
    "CNN (both)": {"score": model_533.score(X_cnn_complete_test, y_cnn_complete_test), "preds": model_533_predictions, "test_set":y_cnn_complete_test},
}

fig, axn = plt.subplots(3, 3, sharex=True, sharey=True, figsize=(12,10))

for i, ax in enumerate(axn.flat):
    try:
        model_name = list(model_data.keys())[i]
        model = model_data[model_name]
        preds = model["preds"]
        score = model["score"]
        test_set = model["test_set"]

        if score is not None and preds is not None and test_set is not None:
            cm = confusion_matrix(test_set, preds)
            sns.heatmap(cm, linewidths=1, ax=ax, annot=True, fmt='g')
            ax.set_title(model_name + f": {score*100:.2f}%", fontsize=8)
        else:
            ax.set_title(model_name + "No results", fontsize=8)
        continue
    except IndexError:
        pass